In [ ]:
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

In [ ]:
wikipedia_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)

In [ ]:
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = loader.load_and_split(text_splitter=splitter)

In [ ]:
vector_store = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector_store.as_retriever()

In [ ]:
from langchain.tools.retriever import create_retriever_tool

In [ ]:
langsmith_retriever_tool = create_retriever_tool(
    retriever, "langsmith_search", "search for information about langsmith"
)

In [ ]:
from langchain_community.utilities.arxiv import ArxivAPIWrapper
from langchain_community.tools.arxiv.tool import ArxivQueryRun

In [ ]:
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv_tool = ArxivQueryRun(api_wrapper=wikipedia_api_wrapper)

In [ ]:
tools = [wikipedia_tool, langsmith_retriever_tool, arxiv_tool]

In [ ]:
from langchain.agents import create_openai_tools_agent
from langchain_openai import ChatOpenAI

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

In [ ]:
llm = ChatOpenAI()

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)

In [ ]:
from langchain.agents import AgentExecutor

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
response = agent_executor.invoke({"input": "Tell me about langsmith"})
response